In [24]:
import os
import pandas as pd
import numpy as np

## Import data

In [25]:
def read_data_files(base_dir):
    """
    Read all required Parquet files from the specified base directory and return a dictionary containing all dataframes.

    :param base_dir: Path to the base directory, typically the directory of the current script
    :return: Dictionary containing all dataframes
    """
    # Define the path to the data directory
    data_dir = os.path.join(base_dir, '..', 'data')

    # Define file paths
    intervention1_path = os.path.join(data_dir, 'interventions1.parquet')
    intervention2_path = os.path.join(data_dir, 'interventions2.parquet')
    intervention3_path = os.path.join(data_dir, 'interventions3.parquet')
    interventionxl1_path = os.path.join(data_dir, 'interventions_bxl.parquet.gzip')
    interventionxl2_path = os.path.join(data_dir, 'interventions_bxl2.parquet.gzip')
    cad_path = os.path.join(data_dir, 'cad9.parquet.gzip')
    ambulance_path = os.path.join(data_dir, 'ambulance_locations.parquet.gzip')
    aed_path = os.path.join(data_dir, 'aed_locations.parquet.gzip')
    pit_path = os.path.join(data_dir, 'pit_locations.parquet.gzip')
    mug_path = os.path.join(data_dir, 'mug_locations.parquet.gzip')

    # Read all Parquet files
    intervention1 = pd.read_parquet(intervention1_path)
    intervention2 = pd.read_parquet(intervention2_path)
    intervention3 = pd.read_parquet(intervention3_path)
    interventionxl1 = pd.read_parquet(interventionxl1_path)
    interventionxl2 = pd.read_parquet(interventionxl2_path)
    cad = pd.read_parquet(cad_path)
    ambulance = pd.read_parquet(ambulance_path)
    aed = pd.read_parquet(aed_path)
    pit = pd.read_parquet(pit_path)
    mug = pd.read_parquet(mug_path)

    return {
        "intervention1": intervention1,
        "intervention2": intervention2,
        "intervention3": intervention3,
        "interventionxl1": interventionxl1,
        "interventionxl2": interventionxl2,
        "cad": cad,
        "ambulance": ambulance,
        "aed": aed,
        "pit": pit,
        "mug": mug
    }

# Get the current working directory
base_dir = os.getcwd()

# Call the function to read all data
data = read_data_files(base_dir)

# Retrieve each dataframe
intervention1 = data["intervention1"]
intervention2 = data["intervention2"]
intervention3 = data["intervention3"]
interventionxl1 = data["interventionxl1"]
interventionxl2 = data["interventionxl2"]
cad = data["cad"]
ambulance = data["ambulance"]
aed = data["aed"]
pit = data["pit"]
mug = data["mug"]


In [26]:
#combine intervention 1,2,3
intervention = pd.concat([intervention1, intervention2, intervention3], axis=0, ignore_index=True)
intervention.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601881 entries, 0 to 601880
Data columns (total 46 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Mission ID                        601881 non-null  int64  
 1   Service Name                      601674 non-null  object 
 2   PostalCode permanence             599280 non-null  float64
 3   CityName permanence               599277 non-null  object 
 4   StreetName permanence             601671 non-null  object 
 5   HouseNumber permanence            35410 non-null   object 
 6   Latitude permanence               507600 non-null  float64
 7   Longitude permanence              587382 non-null  float64
 8   Permanence short name             601843 non-null  object 
 9   Permanence long name              601843 non-null  object 
 10  Vector type                       601881 non-null  object 
 11  EventType Firstcall               601865 non-null  o

In [27]:
#number of unique values each feature
features = intervention.columns
for feature in features:
    print(f'{feature}---->{intervention[feature].nunique()}')

Mission ID---->491158
Service Name---->478
PostalCode permanence---->278
CityName permanence---->327
StreetName permanence---->456
HouseNumber permanence---->23
Latitude permanence---->430
Longitude permanence---->719
Permanence short name---->631
Permanence long name---->632
Vector type---->10
EventType Firstcall---->141
EventLevel Firstcall---->10
EventType Trip---->70
EventLevel Trip---->10
PostalCode intervention---->96
CityName intervention---->3508
Latitude intervention---->102550
Longitude intervention---->190784
Province intervention---->10
T0---->487985
T1---->545684
T1confirmed---->474276
T2---->472774
T3---->452478
T4---->359776
T5---->343052
T6---->393773
T7---->436071
T9---->8910
Intervention time (T1Reported)---->357
Intervention time (T1Confirmed)---->356
Waiting time---->512
Intervention duration---->545
Departure time (T1Reported)---->226
Departure time (T1Confirmed)---->222
Unavailable time---->1861
Name destination hospital---->198
PostalCode destination hospital----

## Pre-process data

In [28]:
## process T3-T0 to get the variable T3-T0

# Process intervention (without specified UTC)
# Convert time columns to datetime format, specifying the format explicitly
intervention['T0'] = pd.to_datetime(intervention['T0'], format='%d%b%y:%H:%M:%S')
intervention['T3'] = pd.to_datetime(intervention['T3'], format='%Y-%m-%d %H:%M:%S.%f')

# Calculate time difference
intervention['Time Difference'] = intervention['T3'] - intervention['T0']
# Convert time difference to minutes
intervention['T3-T0 in Minutes'] = intervention['Time Difference'].dt.total_seconds() / 60

# Process interventionxl1 (with specified UTC)
# Convert time columns to datetime format, handling time zones automatically to UTC
interventionxl1['t0'] = pd.to_datetime(interventionxl1['t0'], utc=True)
interventionxl1['t3'] = pd.to_datetime(interventionxl1['t3'], utc=True)

interventionxl1['Time Difference'] = interventionxl1['t3'] - interventionxl1['t0']
interventionxl1['T3-T0 in Minutes'] = interventionxl1['Time Difference'].dt.total_seconds() / 60

# Process interventionxl2 (without specified UTC)
# Convert time columns to datetime format, specifying the format explicitly
interventionxl2['T0'] = pd.to_datetime(interventionxl2['T0'], format="%d%b%y:%H:%M:%S")
interventionxl2['T3'] = pd.to_datetime(interventionxl2['T3'], format="%d%b%y:%H:%M:%S")

interventionxl2['Time Difference'] = interventionxl2['T3'] - interventionxl2['T0']
interventionxl2['T3-T0 in Minutes'] = interventionxl2['Time Difference'].dt.total_seconds() / 60


In [29]:
## Select features and combine intervention and interventionx1

# Rename columns in xl1 and merge intervention and xl1
interventionxl1.rename(columns={
    'mission_id': 'Mission ID', 
    'postalcode_permanence': 'PostalCode permanence', 
    'eventtype_trip': 'EventType Trip', 
    'eventlevel_trip': 'EventLevel Trip',
    'postalcode_intervention': 'PostalCode intervention',
    'latitude_intervention': 'Latitude intervention',
    'longitude_intervention': 'Longitude intervention',
    'abandon_reason': 'Abandon reason',
    't0': 'T0',
    't3': 'T3'
}, inplace=True)

columns_to_use1 = [
    'Mission ID', 'PostalCode permanence', 'EventType Trip', 
    'EventLevel Trip', 'Latitude intervention', 'Longitude intervention', 
    'T3-T0 in Minutes', 'T0', 'T3', 'Abandon reason'
]

intervention_selected = intervention[columns_to_use1]
interventionxl1_selected = interventionxl1[columns_to_use1]

intervention_combine1 = pd.concat([intervention_selected, interventionxl1_selected], axis=0, ignore_index=True)

intervention_combine1.head(10)
intervention_combine1.info()

# Process EventType Trip, keep only the code of the type
eventtype_1 = intervention_combine1['EventType Trip'].str.split(' ', expand=True)
eventtype_1 = pd.DataFrame(eventtype_1)
intervention_combine1.loc[:, 'EventType Trip'] = eventtype_1[0]

# Function to check unique values of a specific variable
def unique_values_of_column(series):
    """Return all unique values in the Series."""
    return series.unique()

# Use the modified function to get and print unique values of the "Latitude intervention" column
unique = unique_values_of_column(intervention_combine1['Latitude intervention'])
print(unique)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717528 entries, 0 to 717527
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Mission ID              717528 non-null  int64  
 1   PostalCode permanence   714927 non-null  float64
 2   EventType Trip          717428 non-null  object 
 3   EventLevel Trip         703419 non-null  object 
 4   Latitude intervention   626805 non-null  float64
 5   Longitude intervention  716940 non-null  float64
 6   T3-T0 in Minutes        538753 non-null  float64
 7   T0                      717528 non-null  object 
 8   T3                      538753 non-null  object 
 9   Abandon reason          104902 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 54.7+ MB
[5.112496e+01 5.130626e+01 5.123878e+01 ... 5.086053e+06 5.077876e+06
 5.090470e+05]


In [30]:
## Process interventionxl2

# Select the required variables
columns_to_use_xl2 = [
    'Mission ID', 'Cityname Intervention', 'Longitude intervention', 
    'Latitude intervention', 'EventType and EventLevel', 
    'Cityname Permanence', 'T3-T0 in Minutes', 'T0', 'T3', 'Abandon reason NL'
]

interventionxl2_selected1 = interventionxl2[columns_to_use_xl2]

interventionxl2_selected1.head(10)
interventionxl2_selected1.info()

# Process cityname, original format: postcode+cityname, generate new column PostalCode permanence
cityname_P = interventionxl2_selected1['Cityname Permanence'].str.split(' ', expand=True)
cityname_P = pd.DataFrame(cityname_P)
cityname_P[0] = pd.to_numeric(cityname_P[0], errors='coerce', downcast="integer")
interventionxl2_selected1.loc[:, 'PostalCode permanence'] = cityname_P[0]

# Process to generate new column PostalCode intervention
cityname_i = interventionxl2_selected1['Cityname Intervention'].str.split(' ', expand=True)
cityname_i = pd.DataFrame(cityname_i)
cityname_i[0] = pd.to_numeric(cityname_i[0], errors='coerce', downcast="integer")
interventionxl2_selected1.loc[:, 'PostalCode intervention'] = cityname_i[0]

# Process EventType and EventLevel, original format Type+Level+event
event = interventionxl2_selected1['EventType and EventLevel'].str.split(' ', expand=True)
event = pd.DataFrame(event)
interventionxl2_selected1.loc[:, 'EventType Trip'] = event[0]
interventionxl2_selected1.loc[:, 'EventLevel Trip'] = event[1]

# Process abandon reason
interventionxl2_selected1.rename(columns={'Abandon reason NL': 'Abandon reason'}, inplace=True)

interventionxl2_selected1.head(10)
interventionxl2_selected1.info()

# Extract the required columns to match the previous tables and merge all tables
columns_to_use1 = [
    'Mission ID', 'PostalCode permanence', 'EventType Trip', 'EventLevel Trip', 
    'Latitude intervention', 'Longitude intervention', 'T3-T0 in Minutes', 
    'T0', 'T3', 'Abandon reason'
]

interventionxl2_selected2 = interventionxl2_selected1[columns_to_use1]

interventionxl2_selected2.head(10)
interventionxl2_selected2.info()

# Use the modified function to get and print unique values
unique = unique_values_of_column(interventionxl2_selected2['EventLevel Trip'])
print(unique)

# Check the number of unique values
features = interventionxl2_selected1.columns
for feature in features:
    print(f'{feature} ----> {interventionxl2_selected1[feature].nunique()}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38620 entries, 0 to 38619
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Mission ID                38620 non-null  int64         
 1   Cityname Intervention     38620 non-null  object        
 2   Longitude intervention    38609 non-null  float64       
 3   Latitude intervention     38609 non-null  float64       
 4   EventType and EventLevel  36679 non-null  object        
 5   Cityname Permanence       34320 non-null  object        
 6   T3-T0 in Minutes          16874 non-null  float64       
 7   T0                        38620 non-null  datetime64[ns]
 8   T3                        16874 non-null  datetime64[ns]
 9   Abandon reason NL         4294 non-null   object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(4)
memory usage: 2.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38620 entries, 0

/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/1354401682.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interventionxl2_selected1.loc[:, 'PostalCode permanence'] = cityname_P[0]
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/1354401682.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interventionxl2_selected1.loc[:, 'PostalCode intervention'] = cityname_i[0]
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/1354401682.py:30: SettingWithCopyWarnin

In [31]:
## Merge the first 5 intervention tables
intervention_all_combined = pd.concat([intervention_combine1, interventionxl2_selected2], axis=0, ignore_index=True)

intervention_all_combined.head(10)
intervention_all_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756148 entries, 0 to 756147
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Mission ID              756148 non-null  int64  
 1   PostalCode permanence   749247 non-null  float64
 2   EventType Trip          754107 non-null  object 
 3   EventLevel Trip         740098 non-null  object 
 4   Latitude intervention   665414 non-null  float64
 5   Longitude intervention  755549 non-null  float64
 6   T3-T0 in Minutes        555627 non-null  float64
 7   T0                      756148 non-null  object 
 8   T3                      555627 non-null  object 
 9   Abandon reason          109196 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 57.7+ MB


In [32]:
## Process the merged table

# Filter rows where EventType Trip is P003 (Cardiac arrest)
intervention_all = intervention_all_combined[intervention_all_combined['EventType Trip'] == 'P003']
#intervention_all = intervention_all_combined[intervention_all_combined['EventType Trip'].isin(['P003', 'P039'])]

# Remove rows with abnormal PostalCode permanence values, such as postcodes longer than 4 digits (e.g., 59300)
ids_to_remove_postcode = [50221590180, 50221620575, 50221630543, 80221550419, 80221570734]
intervention_all = intervention_all[~intervention_all['Mission ID'].isin(ids_to_remove_postcode)]

# Remove rows where EventLevel Trip values are "Buitendienststelling" or "Interventieplan"
values_to_remove_level = ['Buitendienststelling', 'Interventieplan']
intervention_all = intervention_all[~intervention_all['EventLevel Trip'].isin(values_to_remove_level)]

# Handle inconsistent EventLevel Trip values like N1 and N01
# Create a mapping dictionary
replacements = {
    'N01': 'N1',
    'N02': 'N2',
    'N03': 'N3',
    'N04': 'N4',
    'N05': 'N5',
    'N06': 'N6',
    'N07': 'N7',
    'N08': 'N8',
    'N7A': 'N7',
    'N7B': 'N7',
}
intervention_all['EventLevel Trip'] = intervention_all['EventLevel Trip'].replace(replacements)

# Remove empty values
# Replace empty strings with NaN in EventLevel Trip
intervention_all['EventLevel Trip'] = intervention_all['EventLevel Trip'].replace('', np.nan)

# Specify columns to check for NaN values
columns_to_check = intervention_all.columns.difference(['Abandon reason'])
# Drop rows with NaN values in columns other than 'Abandon reason'
intervention_all = intervention_all.dropna(subset=columns_to_check)

# Handle duplicate Mission ID values
# Function to extract number from a string
def extract_number(s):
    if len(s) > 1:  # Ensure the string is long enough to avoid indexing errors
        try:
            return int(s[1:])  # Try to extract and convert the number
        except ValueError:
            return float('inf')  # Return a large number if conversion fails
    return float('inf')  # Return a large number if the string format is unexpected

# Function to process groups of duplicate Mission IDs
def process_group(group):
    sorted_group = group.sort_values(by='Abandon reason', key=lambda col: col.notna(), ascending=False)
    if sorted_group['Abandon reason'].notna().any():
        return sorted_group.head(1)
    sorted_group['EventLevel number'] = sorted_group['EventLevel Trip'].apply(extract_number)
    sorted_group = sorted_group.sort_values(by='EventLevel number')
    min_event_level = sorted_group['EventLevel number'].min()
    duplicates = sorted_group[sorted_group['EventLevel number'] == min_event_level]
    if len(duplicates) > 1:
        return duplicates.sample(n=1)  # Randomly select one if there are duplicate minimum numbers
    return duplicates.head(1)

# Apply the processing function
intervention_all = intervention_all.groupby('Mission ID').apply(process_group).reset_index(drop=True)
intervention_all = intervention_all.drop(columns=['EventLevel number'])

# Handle missing decimal points in coordinate values
# Function to format latitude values
def format_latitude(lat):
    lat_str = str(lat)
    if '.' in lat_str:
        lat_str = lat_str.replace('.', '')  # Remove the original decimal point
    if len(lat_str) > 2:
        lat_str = lat_str[:2] + '.' + lat_str[2:8]
    return float(lat_str)

# Function to format longitude values
def format_longitude(lon):
    lon_str = str(lon)
    if '.' in lon_str:
        lon_str = lon_str.replace('.', '')  # Remove the original decimal point
    if len(lon_str) > 1:
        lon_str = lon_str[:1] + '.' + lon_str[1:5]
    return float(lon_str)

# Convert values greater than 60 in the Latitude intervention column to a two-digit number
intervention_all['Latitude intervention'] = intervention_all['Latitude intervention'].apply(
    lambda x: format_latitude(x) if x > 60 else x
)

# Convert values greater than 8 in the Longitude intervention column to a single-digit number
intervention_all['Longitude intervention'] = intervention_all['Longitude intervention'].apply(
    lambda x: format_longitude(x) if x > 8 else x
)

# Add a new target column
intervention_all['target'] = intervention_all['Abandon reason'].apply(
    lambda x: 1 if x in ['Overleden', 'Dood Ter Plaatse'] else 0
)


In [33]:
## Check for missing values
intervention_all.info()

missing_values_count = intervention_all.isnull().sum()
print(missing_values_count)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3689 entries, 0 to 3688
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Mission ID              3689 non-null   int64  
 1   PostalCode permanence   3689 non-null   float64
 2   EventType Trip          3689 non-null   object 
 3   EventLevel Trip         3689 non-null   object 
 4   Latitude intervention   3689 non-null   float64
 5   Longitude intervention  3689 non-null   float64
 6   T3-T0 in Minutes        3689 non-null   float64
 7   T0                      3689 non-null   object 
 8   T3                      3689 non-null   object 
 9   Abandon reason          2349 non-null   object 
 10  target                  3689 non-null   int64  
dtypes: float64(4), int64(2), object(5)
memory usage: 317.2+ KB
Mission ID                   0
PostalCode permanence        0
EventType Trip               0
EventLevel Trip              0
Latitude interve

In [34]:
## Export to Excel

# Parse datetime strings, considering time zones
intervention_all['T0'] = pd.to_datetime(intervention_all['T0'], format='%Y-%m-%d %H:%M:%S', utc=True)
intervention_all['T3'] = pd.to_datetime(intervention_all['T3'], format='%Y-%m-%d %H:%M:%S', utc=True)

# Remove time zone information from datetime columns
intervention_all['T0'] = intervention_all['T0'].dt.tz_localize(None)
intervention_all['T3'] = intervention_all['T3'].dt.tz_localize(None)

# export and download
#data_dir = os.path.join(base_dir, '..', 'data')
#combined_file_path = os.path.join(data_dir, 'intervention_all.xlsx')
#intervention_all.to_excel(combined_file_path, index=False)


## Select cities

In [35]:
brussels_zip_codes = [
    1000, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1081, 1082, 1083,
    1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190,
    1200, 1210
]

antwerp_zip_codes = [
        2000, 2018, 2020, 2030, 2040, 2050, 2060, 2100, 2140, 2170, 2180,
        2600, 2610
    ]

liege_zip_codes = [
    4000, 4020, 4030, 4031, 4032, 4040, 4041,
    4042, 4050, 4051, 4100, 4101, 4102, 4120,
    4121, 4140, 4141, 4150, 4151, 4160, 4161,
    4170, 4171, 4180, 4181, 4190, 4210, 4211,
    4212, 4217, 4218, 4219, 4280, 4287
]


In [36]:
brussels_intervention_data = intervention_all[intervention_all['PostalCode permanence'].isin(brussels_zip_codes)]
antwerp_intervention_data = intervention_all[intervention_all['PostalCode permanence'].isin(antwerp_zip_codes)]
liege_intervention_data = intervention_all[intervention_all['PostalCode permanence'].isin(liege_zip_codes)]

In [37]:
# Extract month information
brussels_intervention_data['Month'] = brussels_intervention_data['T0'].dt.to_period('M')
antwerp_intervention_data['Month'] = antwerp_intervention_data['T0'].dt.to_period('M')
liege_intervention_data['Month'] = liege_intervention_data['T0'].dt.to_period('M')


/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/3571673923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brussels_intervention_data['Month'] = brussels_intervention_data['T0'].dt.to_period('M')
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/3571673923.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antwerp_intervention_data['Month'] = antwerp_intervention_data['T0'].dt.to_period('M')
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/3571673923.py:4

In [38]:
# Function to calculate and display statistics
def display_statistics(data, city_name):
    total_cases = len(data)
    deaths = data['target'].sum()
    survival_rate = ((total_cases - deaths) / total_cases) * 100
    death_rate = (deaths / total_cases) * 100
    
    print(f"Statistics for {city_name}:")
    print(f"Total Cases: {total_cases}")
    print(f"Deaths: {deaths}")
    print(f"Survival Rate: {survival_rate:.2f}%")
    print(f"Death Rate: {death_rate:.2f}%")
    print()

# Display statistics for each city
display_statistics(brussels_intervention_data, "Brussels")
display_statistics(antwerp_intervention_data, "Antwerp")
display_statistics(liege_intervention_data, "Liege")

Statistics for Brussels:
Total Cases: 237
Deaths: 124
Survival Rate: 47.68%
Death Rate: 52.32%

Statistics for Antwerp:
Total Cases: 420
Deaths: 210
Survival Rate: 50.00%
Death Rate: 50.00%

Statistics for Liege:
Total Cases: 336
Deaths: 207
Survival Rate: 38.39%
Death Rate: 61.61%



In [39]:
# Function to calculate statistics by month
def calculate_statistics(data, city_name):
    grouped_data = data.groupby('Month')
    stats_list = []

    for month, group in grouped_data:
        total_cases = len(group)
        deaths = group['target'].sum()
        death_rate = (deaths / total_cases) * 100

        stats_list.append({
            'City': city_name,
            'Month': month,
            'Total Cases': total_cases,
            'Deaths': deaths,
            'Death Rate (%)': death_rate
        })

    return stats_list

# Calculate statistics for each city
brussels_stats = calculate_statistics(brussels_intervention_data, "Brussels")
antwerp_stats = calculate_statistics(antwerp_intervention_data, "Antwerp")
liege_stats = calculate_statistics(liege_intervention_data, "Liege")

# Combine all statistics into a single DataFrame
all_stats = brussels_stats + antwerp_stats + liege_stats
all_stats = pd.DataFrame(all_stats)

# Save the statistics to a file
#combined_file_path = os.path.join(data_dir, 'all_stats.xlsx')
#all_stats.to_excel(combined_file_path, index=False)


In [40]:
#combined_file_path = os.path.join(data_dir, 'brussels_intervention_data.xlsx')
#brussels_intervention_data.to_excel(combined_file_path, index=False)

#combined_file_path = os.path.join(data_dir, 'antwerp_intervention_data.xlsx')
#antwerp_intervention_data.to_excel(combined_file_path, index=False)

#combined_file_path = os.path.join(data_dir, 'liege_intervention_data.xlsx')
#liege_intervention_data.to_excel(combined_file_path, index=False)


In [41]:
# Add a new column 'city'
brussels_intervention_data['city'] = 'Brussels'
antwerp_intervention_data['city'] = 'Antwerp'
liege_intervention_data['city'] = 'Liege'

# Combine DataFrames
citycombined_data = pd.concat([brussels_intervention_data, antwerp_intervention_data, liege_intervention_data], ignore_index=True)

# Export the combined DataFrame to a new Excel file
#combined_file_path = '/Users/zenghui/Downloads/citycombined_intervention_data.xlsx'
#citycombined_data.to_excel(combined_file_path, index=False)


/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/157082825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brussels_intervention_data['city'] = 'Brussels'
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/157082825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antwerp_intervention_data['city'] = 'Antwerp'
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/157082825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

## AED filering

In [42]:
# Get the current working directory
base_dir = os.getcwd()

# Define the path to the data directory
data_dir = os.path.join(base_dir, '..', 'data')

# Define the path to the aed_location.xlsx file
aed_location = os.path.join(data_dir, 'AED_locations.xlsx')

# Read the aed_location.xlsx file
aed_location = pd.read_excel(aed_location)

# Verify the contents
print(aed_location.head())


   Unnamed: 0     id type              address  number  postal_code  \
0           0   13.0  AED  Blvd. Fr. Roosevelt    24.0       7060.0   
1          13   95.0  AED          Peperstraat     2.0       3300.0   
2          17  100.0  AED   Rue Pietro Ferrero     5.0       6700.0   
3          19  102.0  AED          Nieuwe Weg      1.0       2070.0   
4          21  115.0  AED        Gemeenteplein     1.0       3798.0   

             municipality        province  \
0                SOIGNIES         Hainaut   
1                  TIENEN  Vlaams-Brabant   
2                   ARLON      Luxembourg   
3             ZWIJNDRECHT       Antwerpen   
4  Voeren  s Gravenvoeren         Limburg   

                                    location public available hours  \
0                                        NaN    yes       NaN   NaN   
1        Peperstraat 2, 3300 Tienen, Belgium    yes       NaN   NaN   
2  Rue Pietro Ferrero 5, 6700 Arlon, Belgium     no       NaN   NaN   
3    Nieuwe Weg 1,

In [43]:
## filter longitude and latitude
NORTH = 51.51
SOUTH = 49.50
EAST = 6.40
WEST = 2.54
LONG_MIN = 2.4493560308402667
LONG_MAX = 4.020383765257329

aed_locationb = aed_location[
    (aed_location['latitude'] <= NORTH) &
    (aed_location['latitude'] >= SOUTH) &
    (aed_location['longitude'] <= EAST) &
    (aed_location['longitude'] >= WEST) &
    ~((aed_location['longitude'] > LONG_MIN) & (aed_location['longitude'] < LONG_MAX))
]

aed_locationb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 1 to 8078
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    5519 non-null   int64  
 1   id            5518 non-null   float64
 2   type          5519 non-null   object 
 3   address       5519 non-null   object 
 4   number        4880 non-null   float64
 5   postal_code   5519 non-null   float64
 6   municipality  5477 non-null   object 
 7   province      5516 non-null   object 
 8   location      5519 non-null   object 
 9   public        5519 non-null   object 
 10  available     2920 non-null   object 
 11  hours         708 non-null    object 
 12  full_address  5519 non-null   object 
 13  latitude      5519 non-null   float64
 14  longitude     5519 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 689.9+ KB


In [44]:
brussels_aed_data = aed_locationb[aed_locationb['postal_code'].isin(brussels_zip_codes)]
antwerp_aed_data = aed_locationb[aed_locationb['postal_code'].isin(antwerp_zip_codes)]
liege_aed_data = aed_locationb[aed_locationb['postal_code'].isin(liege_zip_codes)]

In [45]:
# Add a new column 'city'
brussels_aed_data['city'] = 'Brussels'
antwerp_aed_data['city'] = 'Antwerp'
liege_aed_data['city'] = 'Liege'

# Combine DataFrames
cityaed_data = pd.concat([brussels_aed_data, antwerp_aed_data, liege_aed_data], ignore_index=True)

# Export the combined DataFrame to a new Excel file
#cityaed_path = os.path.join(data_dir, 'cityaed_data.xlsx')
#cityaed_data.to_excel(cityaed_path, index=False)


/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/3636270512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brussels_aed_data['city'] = 'Brussels'
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/3636270512.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antwerp_aed_data['city'] = 'Antwerp'
/var/folders/dr/0j9y6x3n3nv4s_4ssbwb3g240000gn/T/ipykernel_90634/3636270512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us